In [16]:
import numpy as np
import cv2

In [22]:
def to_grayscale(img):
    height, width, _ = img.shape
    grayscale_img = np.zeros((height, width), dtype=np.uint8)

    for i in range(height):
        for j in range(width):
            pixel = img[i, j]
            gray_value = 0.11 * pixel[0] + 0.53 * pixel[1] + 0.36 * pixel[2]
            grayscale_img[i, j] = gray_value

    return grayscale_img


def otsu(grayscale_img):
    height, width = grayscale_img.shape
    histogram = np.zeros(256)

    for i in range(height):
        for j in range(width):
            pixel = grayscale_img[i, j]
            histogram[pixel] += 1

    pixel_amount = height * width
    probs = histogram / pixel_amount

    total_variances = []
    for i in range(256):
        q1 = sum(probs[:i])
        q2 = sum(probs[i + 1:])

        if q1 == 0 or q2 == 0:
            continue

        mu1 = sum([(x * probs[x]) / q1 for x in range(i)])
        mu2 = sum([(x * probs[x]) / q2 for x in range(i + 1, 256)])

        variance_1 = sum([((x - mu1) ** 2) * (probs[x] / q1) for x in range(i)]) ** 2
        variance_2 = sum([((x - mu2) ** 2) * (probs[x] / q2) for x in range(i + 1, 256)]) ** 2

        total_variances.append((q1 * variance_1 + q2 * variance_2) ** 2)

    return total_variances.index(min(total_variances))


def create_binary_mask(grayscale_img, threshold):
    height, width = grayscale_img.shape
    for i in range(height):
        for j in range(width):
            if grayscale_img[i, j] < threshold:
                grayscale_img[i, j] = 1
            else:
                grayscale_img[i, j] = 0

    return grayscale_img

In [35]:
image_path = 'data/text_img3.png'
image = cv2.imread(image_path)

In [36]:
t = otsu(to_grayscale(image))
mask = create_binary_mask(to_grayscale(image), t)
mask = np.array(mask * 255, dtype=np.uint8)
colored_mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
masked_image = cv2.bitwise_and(image, colored_mask)
cv2.imwrite('result.png', masked_image)

True